In [ ]:
# 04_backtesting.ipynb

# Install necessary libraries
!pip install bt matplotlib

# Import libraries
import pandas as pd
import bt
import matplotlib.pyplot as plt
import joblib

# Load the processed data (features)
data = pd.read_csv("data/processed/combined_features.csv")

# Load the saved model (XGBoost in this case)
xgb_model = joblib.load('models/xgb_model.pkl')

# Prepare data for backtesting: extract closing prices and signals
data['predicted_signal'] = xgb_model.predict(data[['close', 'rsi', 'macd', 'macd_signal', 'ema', 'sma', 'sentiment', 'bitcoin_trend', 'eth_trend', 'bnb_trend']])

# Convert signals to Buy (1), Hold (0), Sell (-1)
data['signal'] = data['predicted_signal'].map({0: -1, 1: 0, 2: 1})

# Create a price series for backtesting
price_series = data['close']

# Define the backtest strategy
def backtest_strategy(price_series, signals):
    # Buy on signal = 1 (Buy), Sell on signal = -1 (Sell)
    strategy = bt.Strategy('crypto_trading',
                           [bt.algos.WeighTarget(signals),
                            bt.algos.Rebalance()])
    
    portfolio = bt.Backtest(strategy, price_series)
    return portfolio

# Define the trading signal (for backtesting)
signals = data['signal']

# Run the backtest
portfolio = backtest_strategy(price_series, signals)

# Create the backtest results
results = bt.run(portfolio)

# Plot the results
results.plot(title="Backtest Performance: Crypto Trading Strategy")
plt.show()

# Print the performance summary
print(results.display())

# Save the backtest results
results.to_csv('reports/backtest_results.csv')

# Output summary
print(f"Backtesting Complete at {datetime.now()}")
